In [4]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

tfd = tfp.distributions
tfb = tfp.bijectors
DTYPE=tf.float32

from tensorflow import keras
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import Progbar
tf.keras.backend.set_floatx('float32')


In [5]:
#Neural network layer with autoregressive matrix
#I suggest to read the documentation on tfb.AutoregressiveNetwork
class NN(Layer):
  
  def __init__(self, input_dim,name = 'NN'):
    super(NN,self).__init__(name = name)
    self.layer=tfb.AutoregressiveNetwork(params=5,activation='relu',event_shape=input_dim, hidden_units=[2048])
    
  def call(self,x):
    input = x
    x = self.layer(x)
    #here we basically assume two normal distributions (see pdf attached)
    #you can have many gaussians if you want but in my experience it makes things worse
    #alpha is the mixing parameter between two gaussians 
    #for more gaussians The layer output (now given by x) can be feeded into 
    #the softmax layer and layers for parameters. (i.e.) params=2*Ng+(Ng) 
    #where 2Ng - log_scale and means of Ng gaussians
    #Ng is a set of mixing parameters to feed into the softmax function
    loc_1,loc_2,scale_1,scale_2,alpha = tf.split(x,5,-1)
    #4 is to cover the presumable range of x mean is set to .4 just so the gaussians arent on top of each other
    loc_1 = 4*tf.nn.tanh(loc_1)+0.4
    loc_2 = 4*tf.nn.tanh(loc_2)-0.4
    scale_1 = tf.nn.sigmoid(scale_1)
    scale_2 = tf.nn.sigmoid(scale_2)
    alpha = tf.nn.sigmoid(alpha)
    out = tf.concat([loc_1,loc_2,scale_1,scale_2,alpha],axis=-1)
    return out

In [6]:
input_dim=100
input_shape=(input_dim,)
inputs = keras.Input(shape=input_shape)
#a very simple autoregresive model (5 parameters from 1 layer)
x = tf.keras.layers.experimental.preprocessing.Rescaling(.5)(inputs)
out = NN(input_dim)(x) 
MADE_V = keras.Model(inputs, out)

In [8]:
#Note that it is very slow

In [10]:
#basically the same energy as before
@tf.function 
def Energy(y):
        lamd = 1.
        f = np.sqrt(2.)
        a = 0.1
        m = 0.5
        kinetic_energy = tf.reduce_sum( m / ( 2 * a ) * (y-tf.roll(y,shift=-1,axis=1))**2,axis=-1)
        potential_energy = tf.reduce_sum(lamd * ( y**2 - f**2 )**2,axis=-1)
        return kinetic_energy + a*potential_energy


In [11]:
input_size=100
#Sampling configurations
def NN_Sample(sample_size):
    configs = np.zeros((sample_size,input_size),dtype=np.float32)
    params = np.zeros((sample_size,input_size,5),dtype=np.float32)
    for cell in np.arange(input_size):
        params[:,cell,:] = MADE_V(configs)[:,cell,:]
        shifts_1,shifts_2, scales_1,scales_2,mix = tf.unstack(params[:,cell,:],5,axis=-1)
        bimix_gauss2 = tfd.Mixture(
                        cat=tfd.Categorical(probs=tf.stack([mix,1.-mix],axis=-1)),
                        components=[
                          tfd.Normal(loc=shifts_1, scale=scales_1),
                          tfd.Normal(loc=shifts_2, scale=scales_2)])
        configs[:,cell]=bimix_gauss2.sample()
    return configs

In [ ]:
@tf.function
def log_probability(xs,output):
    shift_1,shift_2,scale_1,scale_2,mix = tf.unstack(output,5,axis=-1)
    bimix_gauss = tfd.Mixture(
                        cat=tfd.Categorical(probs=tf.stack([mix,1-mix],axis=-1)),
                        components=[
                          tfd.Normal(loc=shift_1, scale=scale_1),
                          tfd.Normal(loc=shift_2, scale=scale_2)])
    pdf = bimix_gauss.log_prob(xs) 
    return tf.reduce_sum(pdf,axis=-1)
@tf.function
def KL_Divergence(xs,energy):
    out = MADE_V(xs)
    log_p = log_probability(xs,out)
    KL_D = log_p+energy
    return tf.math.reduce_mean(KL_D)
@tf.function
def KL_loss(xs,energy,KL_D):
    out = MADE_V(xs)
    log_p = log_probability(xs,out)
    t = log_p*(energy+0.5*log_p)/tf.abs(KL_D)
    return tf.math.reduce_mean(t)

lr_decay = .1
learning_rate = 1e-5
optimizer = keras.optimizers.Adam(lr=learning_rate)
checkpoint_directory = '/drive/My Drive/saved_models_MADE_21/'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=MADE_V)
status = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_directory))

# Training loop
n_epochs = 20
n_iter = 1000
n_samples = 1024
for epoch in range(n_epochs):
    print('Epoch {:}/{:}'.format(epoch, n_epochs))
    progbar = Progbar(n_iter)
    
    
    losses = []
    for iter in range(n_iter):
        
        xs_m = NN_Sample(n_samples)
        es_m = Energy(xs_m)
        KL_D = KL_Divergence(xs_m,es_m)
        with tf.GradientTape() as ae_tape:
            loss = KL_loss(xs_m,es_m,KL_D)
        
        gradients = ae_tape.gradient(loss, MADE_V.trainable_variables)

        
        losses.append(KL_D)
        progbar.add(1, values=[('loss', KL_D)])

        optimizer.apply_gradients(zip(gradients,MADE_V.trainable_variables))
    print('  Mean loss: {}'.format(np.mean(losses)))
    print('  Var of loss: {}'.format(np.var(losses)))
    checkpoint.save(checkpoint_directory+'ckpt_{}'.format(epoch))    
 

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 0/20
  71/1000 [=>............................] - ETA: 42:37 - loss: 142.6220